## **1. Table `markets`**  
Cette table représente les marchés financiers.  

| **Nom du champ** | **Type** | **Description** |
|-----------------|---------|----------------|
| `id` | `SMALLINT PRIMARY KEY DEFAULT nextval('market_id_seq')` | Identifiant unique du marché (clé primaire). Il est auto-incrémenté avec la séquence `market_id_seq`. |
| `name` | `VARCHAR` | Nom du marché (ex. : "Euronext Paris", "NYSE"). |
| `alias` | `VARCHAR` | Alias du marché, une version abrégée ou alternative du nom. |
| `boursorama` | `VARCHAR` | Code du marché utilisé sur **Boursorama** (une plateforme d’informations financières). |
| `sws` | `VARCHAR` | Probablement un code interne ou une référence à une autre plateforme. |
| `euronext` | `VARCHAR` | Code du marché utilisé sur **Euronext** (une des principales bourses européennes). |

---

## **2. Table `companies`**  
Cette table contient les informations des entreprises cotées en bourse.  

| **Nom du champ** | **Type** | **Description** |
|-----------------|---------|----------------|
| `id` | `SMALLINT PRIMARY KEY DEFAULT nextval('company_id_seq')` | Identifiant unique de l’entreprise (clé primaire). Il est auto-incrémenté avec la séquence `company_id_seq`. |
| `name` | `VARCHAR` | Nom de l’entreprise (ex. : "TotalEnergies", "Apple Inc."). |
| `mid` | `SMALLINT DEFAULT 0` | Identifiant du marché (`id` de la table `markets`) où l’entreprise est cotée. Défaut : `0`. |
| `symbol` | `VARCHAR` | Symbole boursier de l’entreprise (ex. : "AAPL" pour Apple, "BNP" pour BNP Paribas). |
| `isin` | `CHAR(12)` | Code **ISIN** (International Securities Identification Number), un identifiant unique de 12 caractères pour chaque titre financier (ex. : "FR0000120271" pour TotalEnergies). |
| `boursorama` | `VARCHAR` | Code de l’entreprise sur **Boursorama**. |
| `euronext` | `VARCHAR` | Code de l’entreprise sur **Euronext**. |
| `pea` | `BOOLEAN DEFAULT FALSE` | Indique si l’action est éligible au **PEA** (Plan d'Épargne en Actions). Valeurs possibles : `TRUE` (oui) ou `FALSE` (non). |
| `sector1` | `VARCHAR` | Secteur principal de l’entreprise (ex. : "Technologie", "Finance"). |
| `sector2` | `VARCHAR` | Sous-secteur (ex. : "Banques", "Énergies renouvelables"). |
| `sector3` | `VARCHAR` | Niveau plus détaillé du secteur (ex. : "Banque d’investissement", "Pétrole & Gaz"). |

---

## **3. Table `stocks`**  
Cette table enregistre l’historique des prix et des volumes des actions à des moments précis.  

| **Nom du champ** | **Type** | **Description** |
|-----------------|---------|----------------|
| `date` | `TIMESTAMPTZ` | Date et heure précises de l’enregistrement, avec fuseau horaire. |
| `cid` | `SMALLINT` | Identifiant de l’entreprise (`id` de la table `companies`). |
| `value` | `FLOAT4` | Valeur de l’action à ce moment précis. |
| `volume` | `FLOAT4` | Volume d’actions échangées à ce moment précis. |

🔹 **Cette table est utile pour stocker des données intrajournalières (tick par tick, minute par minute, etc.).**  

---

## **4. Table `daystocks`**  
Cette table regroupe les données agrégées des actions pour une journée complète.  

| **Nom du champ** | **Type** | **Description** |
|-----------------|---------|----------------|
| `date` | `TIMESTAMPTZ` | Date du jour boursier. |
| `cid` | `SMALLINT` | Identifiant de l’entreprise (`id` de la table `companies`). |
| `open` | `FLOAT4` | Prix d’ouverture du jour. |
| `close` | `FLOAT4` | Prix de clôture du jour. |
| `high` | `FLOAT4` | Prix le plus haut atteint durant la journée. |
| `low` | `FLOAT4` | Prix le plus bas atteint durant la journée. |
| `volume` | `FLOAT4` | Volume total des échanges de la journée. |
| `mean` | `FLOAT4` | Prix moyen de l’action sur la journée. |
| `std` | `FLOAT4` | Écart type des prix sur la journée (mesure de la volatilité). |

🔹 **Cette table est utile pour l’analyse des tendances journalières et la volatilité des actions.**  

---

## **Résumé général**  
- **markets** → Contient la liste des marchés financiers.  
- **companies** → Liste les entreprises cotées en bourse et leurs infos.  
- **stocks** → Stocke les prix et volumes des actions à des instants précis.  
- **daystocks** → Agrège les données boursières par jour (ouverture, clôture, volatilité, etc.).  


In [1]:
import pandas as pd
import numpy as np
import sklearn
import glob
import time
import re
import timescaledb_model as tsdb
import os

HOME = "/home/dimitri/epita/big_data/project/bourse/data/"

In [2]:
import datetime


def extract_date_hours(path):
    file_name = path.split('/')[-1]  # Extract the file name
    date_str, time_str = file_name.split(' ')[1:3]  # Extract date and time parts
    time_str = ".".join(time_str.split('.')[0:2])
    date_time_str = f"{date_str} {time_str}"
    date_time = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
    return date_time

In [3]:
def merge_df(file_path, df):
    df_tmp = pd.read_pickle(
    file_path,
    )
    # df.reset_index(drop=True, inplace=True)
    df_tmp['date'] = extract_date_hours(file_path)
    return pd.concat([df, df_tmp])    

In [20]:
df = pd.DataFrame()
dir = os.listdir(HOME + "boursorama")
i = 0
for dir_date in dir :
    list_file_path = os.listdir(HOME + "boursorama/" + dir_date)
    for file_path in list_file_path :
        df = merge_df(HOME + "boursorama/" + dir_date + "/" + file_path, df)
        if (i == 100) :
            break
        i+=1
    break

In [16]:
df.head()

,last,volume,symbol,name,date
symbol,,,,,
1rPABBV,58.01,0,1rPABBV,ABBVIE,2019-02-28 13:32:01.512834
1rPAC,37.29,405421,1rPAC,ACCOR,2019-02-28 13:32:01.512834
1rPACNV,46.02,0,1rPACNV,ACCOR,2019-02-28 13:32:01.512834
1rPADP,171.7,22674,1rPADP,ADP,2019-02-28 13:32:01.512834
1rPAF,10.86,3512926,1rPAF,AIR FRANCE - KLM,2019-02-28 13:32:01.512834


In [34]:
# select the name where it start with bnp
df[df["name"].str.startswith("Apple")].head()

,last,volume,symbol,name,date
symbol,,,,,


In [36]:
from timescaledb_model import *
db = TimescaleStockMarketModel("bourse", "jmd", "localhost", "123")


Logs of timescaledb_model go to /tmp/bourse.log
Connection attempt failed: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "jmd"

Connection attempt failed: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "jmd"

Connection attempt failed: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "jmd"

Connection attempt failed: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "jmd"

Connection attempt failed: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "jmd"



Exception: Failed to connect to database after multiple attempts

In [ ]:
# create a new df
df_markets = pd.DataFrame()
# TODO
# df_markets['name'] = ["Boursorama", ""]
# df_markets['alias']
# df_markets['boursorama']
# df_markets['sws']
# df_markets['euronext']

# db.write(df_markets, "markets")

In [35]:
df_companies = df.copy()
# fill with nan
df_companies["isin"] = np.nan
df_companies["boursorama"] = df_companies["symbol"]
df["symbol"].apply(lambda x: x[3: len(x)])
df_companies["mid"] = np.nan # TODO
df_companies["euronext"] = np.nan # TODO
df_companies["pea"] = np.nan # TODO

df_companies["sector1"] = np.nan # TODO
df_companies["sector2"] = np.nan # TODO
df_companies["sector3"] = np.nan # TODO

db.df_write(df_companies, "companies")

NameError: name 'db' is not defined